# Setup

### Imports

In [27]:
# Standard
import math

# Pandas and plotting
import pandas as pd
import numpy as np
from plotnine import *
from scipy.sparse import coo_matrix, hstack

# SK Learn requirements
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer



import warnings
warnings.filterwarnings('ignore')

### Load Data

In [10]:
import os

train_path = os.getcwd() + "\\data\\train.tsv"
train = pd.read_csv(train_path, sep='\t')

# Set random seed for reproducibility
pd.np.random.seed(369)

# Preprocessing
## Parse out individual category levels
Product category names consist of subcategory trees (Men/Tops/T-shirts). There don't appear to be more than 5 levels and the vast majority of them have just 3 levels. Below, we create a variable for each level.

In [11]:
from sklearn import preprocessing

train_x = train[["brand_name", "shipping", "category_name"]]
train_y = train.price

def piece(string, delim, n):
    string = str(string)
    if string.count(delim) < n:
        return pd.np.NaN
    return string.split(delim)[n]

train_x["category1"] = train_x.category_name.map(lambda x: piece(x, "/", 0))
train_x["category2"] = train_x.category_name.map(lambda x: piece(x, "/", 1))
train_x["category3"] = train_x.category_name.map(lambda x: piece(x, "/", 2))
train_x["category4"] = train_x.category_name.map(lambda x: piece(x, "/", 3))
train_x["category5"] = train_x.category_name.map(lambda x: piece(x, "/", 4))
train_x = train_x.drop(columns='category_name')
train_x.head()

,brand_name,shipping,category1,category2,category3,category4,category5
0,NaN,1,Men,Tops,T-shirts,NaN,NaN
1,Razer,0,Electronics,Computers & Tablets,Components & Parts,NaN,NaN
2,Target,1,Women,Tops & Blouses,Blouse,NaN,NaN
3,NaN,1,Home,Home Décor,Home Décor Accents,NaN,NaN
4,NaN,0,Women,Jewelry,Necklaces,NaN,NaN


## Fill in missing data

For now we will be representing any missing data with the string 'zMissing.'

In [12]:
train_x = train_x.fillna('zMissing')
train_x.head()

,brand_name,shipping,category1,category2,category3,category4,category5
0,zMissing,1,Men,Tops,T-shirts,zMissing,zMissing
1,Razer,0,Electronics,Computers & Tablets,Components & Parts,zMissing,zMissing
2,Target,1,Women,Tops & Blouses,Blouse,zMissing,zMissing
3,zMissing,1,Home,Home Décor,Home Décor Accents,zMissing,zMissing
4,zMissing,0,Women,Jewelry,Necklaces,zMissing,zMissing


## Organize item_description

Create a list of all the item_descriptions for which we will create a sparse feature matrix. 

In [21]:
train["item_description"]=train["item_description"].astype(str)
descr_corpus = list(train["item_description"])

# One hot encode
http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features

Create dummy variables out of categorical variables. First we use LabelEncoder to swap out category names with numbers. Then we use OneHotEncoder to pivot out those values in to columns.

**OneHotEncoder**

* Encode categorical integer features using a one-hot aka one-of-K scheme.
* The input to this transformer should be a matrix of integers, denoting the values taken on by categorical (discrete) features.
* The output will be a sparse matrix where each column corresponds to one possible value of one feature.
* It is assumed that input features take on values in the range [0, n_values).
* This encoding is needed for feeding categorical data to many scikit-learn estimators, notably linear models and SVMs with the standard kernels.



## Category encoding

In [13]:
# Pandas is proving to be too slow
# dummies = pd.get_dummies(train_x)
# lets try it with sklearn
encoder = preprocessing.OneHotEncoder()
label_encoder = preprocessing.LabelEncoder()
data_label_encoded = train_x.apply(label_encoder.fit_transform)
train_x_encoded = encoder.fit_transform(data_label_encoded)

## Item description encoding

In [ ]:
vectorizer = CountVectorizer()
descr_encoded = vectorizer.fit_transform(descr_corpus)

## Create encoded sparse feature matrix
Horizontally stack the category encoded matrix with the description encoded matrix

In [31]:
hstack([train_x_encoded,descr_encoded])

<1482535x167850 sparse matrix of type '<class 'numpy.float64'>'
	with 40963938 stored elements in COOrdinate format>

# Supervised Learning

Lets start by experimenting with a simple SGD regressor becasue of how fast it is.
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor

Further down the line we should incorporate matrix factorization as the collinearity and correlations are likely quite high. We'll also want to perform cross validation to pick optimal parameters for SGD.

In [ ]:
# This competition uses root mean squared log error.
# Lets use this to evaluate our models as well.
from sklearn.metrics import make_scorer

# vectorized error calc
def rmsle(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y0), 2)))

rmsle_score = make_scorer(rmsle, greater_is_better=False)

## Stochastic Gradient Descent - SGD
Stochastic gradient descent is a simple yet very efficient approach to fit linear models. It is particularly useful when the number of samples (and the number of features) is very large. The partial_fit method allows only/out-of-core learning. The classes SGDClassifier and SGDRegressor provide functionality to fit linear models for classification and regression using different (convex) loss functions and different penalties.

### Grid Search
SGD has a lot of parameters that need to be optimized. Lets try a 10-fold-cross-validation on several potential values and figure out whats giving us the best rmsle.

In [ ]:
param_grid = [
  {'loss': ["squared_loss", "huber", "epsilon_insensitive"], 
   'penalty': ['elasticnet'],
   'alpha' : 10.0**-pd.np.arange(1,7),
   'l1_ratio' : [0, .15, .3, .5, .75, 1],
   'learning_rate' : ['constant', 'optimal', 'invscaling']
  }
 ]
sgd_cv = GridSearchCV(estimator=SGDRegressor(), param_grid=param_grid, cv=5, scoring=rmsle_score)
sgd_cv.fit(train_x_encoded, train_y)

In [ ]:
sgd_cv.best_estimator_

### Optimal SGD
We have found our optimal parameters. Lets use them on the whole data set and see how we're doing.

In [ ]:
sgd = SGDRegressor(alpha=1.0000000000000001e-05, average=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=1, learning_rate='optimal',
       loss='epsilon_insensitive', max_iter=None, n_iter=None,
       penalty='elasticnet', power_t=0.25, random_state=None, shuffle=True,
       tol=None, verbose=0, warm_start=False)
scores = cross_val_score(sgd, train_x_encoded, train_y, cv=10, scoring=rmsle_score)

In [ ]:
scores = np.vectorize(abs)(scores)
print("Average CV performance: {0}".format(scores.mean()))

In [ ]:
ggplot(pd.DataFrame({'scores': scores}), aes('scores')) + \
    geom_density() + geom_point(aes(x="scores", y=0), shape=6, size=4) + \
    ggtitle("10 fold cross validation performance on traing set") + \
    ylab("Density") + xlab("RMSLE") + \
    theme_light()